In [3]:
import requests
import pandas as pd
import json
import os

In [4]:
%load_ext dotenv
%dotenv

In [5]:
api_key=os.environ['API_KEY']

In [6]:
demo_groups = ['B01001', 'B05003', 'B02015', 'DP05', 'B03001']

soci_groups = ['B00002', 'B11002', 'B04006', 'B11009', 'B05005', 
                'C16001', 'B05006', 'DP02', 'B07204', 'S1810']
                
econ_groups = ['B00001', 'B19025', 'B00002', 'B19101', 'B01001', 
                'B19201', 'B08013', 'B19313', 'B17010', 'B20005', 
                'B17024', 'DP03', 'B19001', 'S1701']
                
hous_groups = ['B00002', 'B25070', 'B25004', 'B25075', 'B25008', 
              'DP04', 'B25063']

all_groups = demo_groups + soci_groups + econ_groups + hous_groups

In [7]:
groups = json.loads(requests.get('https://api.census.gov/data/2017/acs/acs5/groups.json').content)
profile_groups = json.loads(requests.get('https://api.census.gov/data/2017/acs/acs5/profile/groups.json').content)
cprofile_groups = json.loads(requests.get('https://api.census.gov/data/2017/acs/acs5/cprofile/groups.json').content)
subject_groups = json.loads(requests.get('https://api.census.gov/data/2017/acs/acs5/subject/groups.json').content)

In [8]:
meta = pd.DataFrame(groups['groups']+profile_groups['groups']+cprofile_groups['groups']+subject_groups['groups'])

In [9]:
meta = meta[meta['name'].isin(all_groups)]

In [11]:
meta.head()

,name,description,variables
21,B17010,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILI...,https://api.census.gov/data/2017/acs/acs5/grou...
37,B07204,GEOGRAPHICAL MOBILITY IN THE PAST YEAR FOR CUR...,https://api.census.gov/data/2017/acs/acs5/grou...
68,B05003,SEX BY AGE BY NATIVITY AND CITIZENSHIP STATUS,https://api.census.gov/data/2017/acs/acs5/grou...
70,B05006,PLACE OF BIRTH FOR THE FOREIGN-BORN POPULATION...,https://api.census.gov/data/2017/acs/acs5/grou...
72,B05005,PERIOD OF ENTRY BY NATIVITY AND CITIZENSHIP ST...,https://api.census.gov/data/2017/acs/acs5/grou...


#### Get variable definitions for each column

In [27]:
all_var = []
for i in meta.variables:
    r = json.loads(requests.get(i).content)['variables']
    for key, value in r.items():
        value.update(dict(variable=key))
        all_var.append(value)

In [30]:
df_meta = pd.DataFrame(all_var)
df_meta = df_meta[~df_meta['variable'].str.endswith('A')]

In [31]:
df_meta.head()

,label,predicateType,group,limit,predicateOnly,variable,concept
26,Margin of Error!!Total!!Income in the past 12 ...,int,B17010,0,True,B17010_020M,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILI...
27,Estimate!!Total!!Income in the past 12 months ...,int,B17010,0,True,B17010_020E,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILI...
28,Margin of Error!!Total!!Income in the past 12 ...,int,B17010,0,True,B17010_022M,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILI...
29,Estimate!!Total!!Income in the past 12 months ...,int,B17010,0,True,B17010_022E,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILI...
30,Margin of Error!!Total!!Income in the past 12 ...,int,B17010,0,True,B17010_021M,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILI...


In [40]:
df_meta['label1'] = df_meta['label'].apply(lambda x: x.replace('Margin of Error', '').replace('Estimate', '').replace('!', ''))

In [43]:
df_meta.to_csv('data/meta.csv', index=False)